In [1]:
import ee
import pandas as pd
import geemap

In [2]:
ee.Initialize()

### Define basic parameters

In [3]:
# import the China_zone
China_shp = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")

In [4]:
Target_area_cn = '中南'
Target_area_en = 'zhongnan'

In [5]:
Region = China_shp.filterMetadata('NAME1','equals',Target_area_cn)

### Import all images

In [6]:
# read all asset_df into a list
asset_list = [pd.read_csv(f'./gee_asset_china/{str(i).zfill(2)}_path.csv') for i in range(1,22)]

In [7]:
# concate all asset_df into one df
asset_df = pd.concat(asset_list)

In [8]:
asset_df

,year,type,path,region
0,1990_1992,Fourier,projects/earthengine-legacy/assets/users/Built...,huadong
1,1993_1995,Fourier,projects/earthengine-legacy/assets/users/Built...,huadong
2,1996_1998,Fourier,projects/earthengine-legacy/assets/users/Built...,huadong
3,1999_2001,Fourier,projects/earthengine-legacy/assets/users/Built...,huadong
4,2002_2004,Fourier,projects/earthengine-legacy/assets/users/Built...,huadong
...,...,...,...,...
7,2011_2013,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/rvang...,huabei
8,2014_2016,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/rvang...,huabei
0,2017_2019,Fourier,projects/earthengine-legacy/assets/users/nmoon...,huabei
1,2017_2019,Landsat_cloud_free,projects/earthengine-legacy/assets/users/nmoon...,huabei


### Filter the target image

In [9]:
year = '1990_1992'

In [10]:
landsat    = asset_df[(asset_df['type']=='Landsat_cloud_free') & (asset_df['year']==year)]
normalized = asset_df[(asset_df['type']=='NDVI_NDBI_EVI')      & (asset_df['year']==year)]
fourier    = asset_df[(asset_df['type']=='Fourier')            & (asset_df['year']==year)]

In [11]:
Fourier_VIS = {"opacity":1,"bands":["NDBI_cos_1","EVI_sin_2","NDVI_sin_2"],"min":-106,"max":158,"gamma":1}

In [12]:
landsat_img = ee.ImageCollection([ee.Image(i) for i in landsat['path']]).mosaic().clip(Region)
normalized_img = ee.ImageCollection([ee.Image(i) for i in normalized['path']]).mosaic().clip(Region)
fourier_img = ee.ImageCollection([ee.Image(i) for i in fourier['path']]).mosaic().clip(Region)

In [20]:
normalized_img.bandNames().getInfo()

['NDVI', 'NDBI', 'EVI']

In [21]:
Map = geemap.Map()

Map.add_basemap('HYBRID')
Map.setCenter(107.19, 33.65,5)

Map.addLayer(landsat_img,   {'bands':['B4','B3','B5'],'min':0,'max':90},'landsat_img')
Map.addLayer(normalized_img,{'min':-15,'max':30},'normalized_img')
Map.addLayer(fourier_img,    Fourier_VIS,'fourier_img')

Map

Map(center=[33.65, 107.19], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

In [18]:
# crate the export task
task = ee.batch.Export.image.toCloudStorage(
    image = normalized_img,
    description=f'normalized_img_{year}_{Target_area_en}',
    bucket='north_china_plain',
    fileNamePrefix=f'China_Built/normalized_img_{year}_{Target_area_en}',
    region=Region.geometry(),
    scale=30,
    maxPixels=int(1e13),
    skipEmptyTiles=True)

task.start()